# 🦎 Youtube Video Downloader 🦎

## 🐍 Encubadora 🐍

In [405]:
# import tkinter as tk
# from tkinter import messagebox, filedialog
# from pytube import YouTube
# import os
# import threading

# def select_download_path():
#     download_path = filedialog.askdirectory()
#     if download_path:
#         download_path_entry.delete(0, tk.END)
#         download_path_entry.insert(0, download_path)
#         # Iniciar uma thread para atualizar as informações de qualidade
#         thread = threading.Thread(target=update_info_in_background)
#         thread.start()

# def update_info_in_background():
#     update_quality_dropdown()

# # Atualizar as opções de qualidade quando o usuário inserir o link do vídeo
# def on_url_entry_change(event):
#     # Iniciar uma thread para atualizar as informações de qualidade
#     thread = threading.Thread(target=update_info_in_background)
#     thread.start()

# def browse_and_update():
#     # Iniciar uma thread para atualizar as informações de qualidade
#     thread = threading.Thread(target=update_info_in_background)
#     thread.start()
#     select_download_path()

# def get_available_qualities():
#     url = url_entry.get()
#     try:
#         yt = YouTube(url)
#         # Filtrar as qualidades disponíveis para evitar duplicatas e remover opções indesejadas
#         available_qualities = set()
#         for stream in yt.streams.filter(type="video"):
#             if stream.resolution not in available_qualities and stream.resolution is not None:
#                 available_qualities.add(stream.resolution)
#         return sorted(available_qualities)
#     except Exception as e:
#         messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
#         return []

# def update_quality_dropdown():
#     try:
#         qualities = get_available_qualities()
#         quality_var.set("")  # Limpar o valor atual do menu suspenso
#         quality_dropdown["menu"].delete(0, "end")  # Limpar as opções do menu suspenso
#         for quality in qualities:
#             quality_dropdown["menu"].add_command(label=quality, command=tk._setit(quality_var, quality))
#     except Exception as e:
#         messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
#         return

# def download_video():
#     url = url_entry.get()
#     selected_quality = quality_var.get()
#     try:
#         yt = YouTube(url)
#         # Obter a stream de vídeo com a qualidade selecionada
#         video = yt.streams.filter(type="video", resolution=selected_quality).first()
#         if video is None:
#             messagebox.showerror("Erro", f"A qualidade '{selected_quality}' não é suportada para este vídeo.")
#             return
#         download_path = download_path_entry.get()
#         file_path = os.path.join(download_path, video.default_filename)
#         video.download(download_path)
        
#         # Notificar o usuário sobre o sucesso do download
#         messagebox.showinfo("Sucesso", f"Vídeo baixado com sucesso! Salvo em: {file_path}")
#     except Exception as e:
#         messagebox.showerror("Erro", f"Ocorreu um erro ao baixar o vídeo: {str(e)}")

# # Criar a janela principal
# root = tk.Tk()
# root.title("YouTube Video Downloader")

# # Criar os widgets
# url_label = tk.Label(root, text="URL do vídeo:")
# url_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)

# url_entry = tk.Entry(root, width=50)
# url_entry.grid(row=0, column=1, padx=10, pady=5)

# download_path_label = tk.Label(root, text="Caminho de Download:")
# download_path_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)

# download_path_entry = tk.Entry(root, width=50)
# download_path_entry.grid(row=1, column=1, padx=10, pady=5)

# browse_button = tk.Button(root, text="Navegar...", command=select_download_path)
# browse_button.grid(row=1, column=2, padx=5, pady=5)

# quality_label = tk.Label(root, text="Qualidade do vídeo:")
# quality_label.grid(row=2, column=0, padx=10, pady=5, sticky=tk.W)

# quality_var = tk.StringVar(root)
# quality_dropdown = tk.OptionMenu(root, quality_var, "")
# quality_dropdown.grid(row=2, column=1, padx=10, pady=5)

# download_button = tk.Button(root, text="Baixar Vídeo", command=download_video)
# download_button.grid(row=3, column=0, columnspan=2, pady=10)

# # Executar a aplicação
# root.mainloop()

## 🐢 Depuração 🐢

### Bibliotecas

In [406]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox, filedialog
from pytube import YouTube
import sys
import os
import threading

### select_download_path

In [407]:
def select_download_path():
    download_path = filedialog.askdirectory()
    if download_path:
        download_path_entry.delete(0, tk.END)
        download_path_entry.insert(0, download_path)
        # Iniciar uma thread para atualizar as informações de qualidade
        thread = threading.Thread(target=update_info_in_background)
        thread.start()

### update_info_in_background

In [408]:
def update_info_in_background():
    update_quality_dropdown()

### on_url_entry_change

In [409]:
# Atualizar as opções de qualidade quando o usuário inserir o link do vídeo
def on_url_entry_change(event):
    # Iniciar uma thread para atualizar as informações de qualidade
    thread = threading.Thread(target=update_info_in_background)
    thread.start()

### browse_and_update

In [410]:
def browse_and_update():
    # Iniciar uma thread para atualizar as informações de qualidade
    thread = threading.Thread(target=update_info_in_background)
    thread.start()
    select_download_path()

### get_available_qualities

In [411]:
def get_available_qualities():
    url = url_entry.get()
    try:
        yt = YouTube(url)
        # Filtrar as qualidades disponíveis para evitar duplicatas e remover opções indesejadas
        available_qualities = set()
        for stream in yt.streams.filter(type="video"):
            if stream.resolution not in available_qualities and stream.resolution is not None:
                available_qualities.add(stream.resolution)
        return sorted(available_qualities)
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
        return []

### update_quality_dropdown

In [412]:
def update_quality_dropdown():
    try:
        qualities = get_available_qualities()
        quality_var.set("")  # Limpar o valor atual do menu suspenso
        quality_dropdown["menu"].delete(0, "end")  # Limpar as opções do menu suspenso
        for quality in qualities:
            quality_dropdown["menu"].add_command(label=quality, command=tk._setit(quality_var, quality))
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
        return

### download_video

In [413]:
def download_video():
    url = url_entry.get()
    selected_quality = quality_var.get()
    try:
        yt = YouTube(url)
        # Obter a stream de vídeo com a qualidade selecionada
        video = yt.streams.filter(type="video", resolution=selected_quality).first()
        if video is None:
            messagebox.showerror("Erro", f"A qualidade '{selected_quality}' não é suportada para este vídeo.")
            return
        download_path = download_path_entry.get()
        file_path = os.path.join(download_path, video.default_filename)
        video.download(download_path)
        
        # Notificar o usuário sobre o sucesso do download
        messagebox.showinfo("Sucesso", f"Vídeo baixado com sucesso! Salvo em: {file_path}")
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao baixar o vídeo: {str(e)}")

### interface

In [414]:
# Criar a janela principal
root = tk.Tk()
root.title("YouTube Video Downloader")

# Definir a cor de fundo da janela
root.configure(bg="black")

# Criar os widgets
url_label = tk.Label(root, text="URL do vídeo:")
url_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
url_label.configure(bg="black")
url_label.configure(fg="white")

url_entry = tk.Entry(root, width=50)
url_entry.grid(row=0, column=1, padx=10, pady=5)
url_entry.configure(bg="black")
url_entry.configure(fg="white")

download_path_label = tk.Label(root, text="Caminho de Download:")
download_path_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
download_path_label.configure(bg="black")
download_path_label.configure(fg="white")

download_path_entry = tk.Entry(root, width=50)
download_path_entry.grid(row=1, column=1, padx=10, pady=5)
download_path_entry.configure(bg="black")
download_path_entry.configure(fg="white")

browse_button = tk.Button(root, text="Navegar...", command=select_download_path)
browse_button.grid(row=1, column=2, padx=5, pady=5)
browse_button.configure(bg="black")
browse_button.configure(fg="white")

quality_label = tk.Label(root, text="Qualidade do vídeo:")
quality_label.grid(row=2, column=0, padx=10, pady=5, sticky=tk.W)
quality_label.configure(bg="black")
quality_label.configure(fg="white")

quality_var = tk.StringVar(root)
quality_var.set("Selecione")  # Definindo um valor padrão

quality_dropdown = tk.OptionMenu(root, quality_var, "")
quality_dropdown.grid(row=2, column=1, padx=10, pady=5)
quality_dropdown.configure(bg="black")
quality_dropdown.configure(fg="white")

download_button = tk.Button(root, text="Baixar Vídeo", command=download_video)
download_button.grid(row=3, column=0, columnspan=2, pady=10)
download_button.configure(bg="black")
download_button.configure(fg="white")

### executa

In [415]:
# Executar a aplicação
root.mainloop()

### 🦖 Old 🦖

In [416]:
# import tkinter as tk
# from tkinter import messagebox, filedialog
# from pytube import YouTube
# import os
# import threading

# def select_download_path():
#     download_path = filedialog.askdirectory()
#     if download_path:
#         download_path_entry.delete(0, tk.END)
#         download_path_entry.insert(0, download_path)
#         # Iniciar uma thread para atualizar as informações de qualidade
#         thread = threading.Thread(target=update_info_in_background)
#         thread.start()

# def update_info_in_background():
#     update_quality_dropdown()

# # Atualizar as opções de qualidade quando o usuário inserir o link do vídeo
# def on_url_entry_change(event):
#     # Iniciar uma thread para atualizar as informações de qualidade
#     thread = threading.Thread(target=update_info_in_background)
#     thread.start()

# def browse_and_update():
#     # Iniciar uma thread para atualizar as informações de qualidade
#     thread = threading.Thread(target=update_info_in_background)
#     thread.start()
#     select_download_path()

# def get_available_qualities():
#     url = url_entry.get()
#     try:
#         yt = YouTube(url)
#         # Filtrar as qualidades disponíveis para evitar duplicatas e remover opções indesejadas
#         available_qualities = set()
#         for stream in yt.streams.filter(type="video"):
#             if stream.resolution not in available_qualities and stream.resolution is not None:
#                 available_qualities.add(stream.resolution)
#         return sorted(available_qualities)
#     except Exception as e:
#         messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
#         return []

# def update_quality_dropdown():
#     try:
#         qualities = get_available_qualities()
#         quality_var.set("")  # Limpar o valor atual do menu suspenso
#         quality_dropdown["menu"].delete(0, "end")  # Limpar as opções do menu suspenso
#         for quality in qualities:
#             quality_dropdown["menu"].add_command(label=quality, command=tk._setit(quality_var, quality))
#     except Exception as e:
#         messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
#         return

# def download_video():
#     url = url_entry.get()
#     selected_quality = quality_var.get()
#     try:
#         yt = YouTube(url)
#         # Obter a stream de vídeo com a qualidade selecionada
#         video = yt.streams.filter(type="video", resolution=selected_quality).first()
#         if video is None:
#             messagebox.showerror("Erro", f"A qualidade '{selected_quality}' não é suportada para este vídeo.")
#             return
#         download_path = download_path_entry.get()
#         file_path = os.path.join(download_path, video.default_filename)
#         video.download(download_path)
        
#         # Notificar o usuário sobre o sucesso do download
#         messagebox.showinfo("Sucesso", f"Vídeo baixado com sucesso! Salvo em: {file_path}")
#     except Exception as e:
#         messagebox.showerror("Erro", f"Ocorreu um erro ao baixar o vídeo: {str(e)}")

# # Criar a janela principal
# root = tk.Tk()
# root.title("YouTube Video Downloader")

# # Criar os widgets
# url_label = tk.Label(root, text="URL do vídeo:")
# url_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)

# url_entry = tk.Entry(root, width=50)
# url_entry.grid(row=0, column=1, padx=10, pady=5)

# download_path_label = tk.Label(root, text="Caminho de Download:")
# download_path_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)

# download_path_entry = tk.Entry(root, width=50)
# download_path_entry.grid(row=1, column=1, padx=10, pady=5)

# browse_button = tk.Button(root, text="Navegar...", command=select_download_path)
# browse_button.grid(row=1, column=2, padx=5, pady=5)

# quality_label = tk.Label(root, text="Qualidade do vídeo:")
# quality_label.grid(row=2, column=0, padx=10, pady=5, sticky=tk.W)

# quality_var = tk.StringVar(root)
# quality_dropdown = tk.OptionMenu(root, quality_var, "")
# quality_dropdown.grid(row=2, column=1, padx=10, pady=5)

# download_button = tk.Button(root, text="Baixar Vídeo", command=download_video)
# download_button.grid(row=3, column=0, columnspan=2, pady=10)

# # Executar a aplicação
# root.mainloop()